In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [10]:
class block(nn.Module):
    def __init__(self, in_channels, intermidate_channels, identity_downsample = None, stride = 1):
        super().__init__()
        self.expantion = 4
        self.conv1 = nn.Conv2d(in_channels,intermidate_channels, kernel_size = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(intermidate_channels)
        self.conv2 = nn.Conv2d(intermidate_channels,intermidate_channels,kernel_size=3, stride = stride, padding = 1, bias = False )
        self.bn2 = nn.BatchNorm2d(intermidate_channels)
        self.conv3 = nn.Conv2d(intermidate_channels, intermidate_channels*4, kernel_size = 1, bias = False )
        self.bn3 = nn.BatchNorm2d(intermidate_channels*4)
        self.identity_downsample = identity_downsample
        self.stride = stride


    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = F.relu(x)
        return x

        

    

In [41]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels,self.in_channels, kernel_size = 7, stride=2, padding= 3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)

        self.layer1 = self._make_layer(block, layers[0], 64, 1)
        self.layer2 = self._make_layer(block, layers[1], 128, 2)
        self.layer3 = self._make_layer(block, layers[2], 256, 2)
        self.layer4 = self._make_layer(block, layers[3], 512, 2)

        self.fc1 = nn.Linear(512*4, num_classes)

    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding =1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = F.adaptive_avg_pool2d(x,(1,1))
        x = x.flatten(1)
        x = self.fc1(x)
        x = F.softmax(x)
        return x


    def _make_layer(self, block, num_residual_blocks, intermidate_channels, stride):
        identity_downsample= None
        layers = []

        if stride != 1 or self.in_channels != (intermidate_channels*4):
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, intermidate_channels*4, stride=stride, kernel_size= 1, bias = False),
                nn.BatchNorm2d(intermidate_channels *4)
            )

        layers.append(
            block(self.in_channels,intermidate_channels, identity_downsample, stride)
            )
        
        self.in_channels = intermidate_channels *4

        for i in range(num_residual_blocks -1):
            layers.append(block(self.in_channels, intermidate_channels))
        
        return nn.Sequential(*layers)


In [53]:
def ResNet50(img_channel = 3, num_classes = 1000):
    return ResNet(block, [3,4,6,3], img_channel, num_classes)



In [58]:
def test():
    BATCH_SIZE = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = ResNet50(img_channel=3, num_classes=1000).to(device)
    y = net(torch.randn(BATCH_SIZE, 3, 224, 224)).to(device)
    assert y.size() == torch.Size([BATCH_SIZE, 1000])
    print(y.size())


if __name__ == "__main__":
    test()

torch.Size([4, 1000])


C:\Users\Marcel\AppData\Local\Temp\ipykernel_32448\318093757.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
